In [1]:
import numpy as np
import pandas as pd
from ahrs.filters import Complementary

def complementary_filter(df):
    # inicialização do filtro complementar, gain -> 1 giroscópio, gain -> 0 acelerômetro
    cf = Complementary(frequency=100, gain=0.75)
    
    # grupo dos sensores, seguindo lógica do gráfico
    sensor_groups = {
        'right_foot': ['accelerometer_right_foot_x', 'accelerometer_right_foot_y', 'accelerometer_right_foot_z', 'gyroscope_right_foot_x', 'gyroscope_right_foot_y', 'gyroscope_right_foot_z'],
        'right_shin': ['accelerometer_right_shin_x', 'accelerometer_right_shin_y', 'accelerometer_right_shin_z', 'gyroscope_right_shin_x', 'gyroscope_right_shin_y', 'gyroscope_right_shin_z'],
        'right_thigh': ['accelerometer_right_thigh_x', 'accelerometer_right_thigh_y', 'accelerometer_right_thigh_z', 'gyroscope_right_thigh_x', 'gyroscope_right_thigh_y', 'gyroscope_right_thigh_z'],
        'left_foot': ['accelerometer_left_foot_x', 'accelerometer_left_foot_y', 'accelerometer_left_foot_z', 'gyroscope_left_foot_x', 'gyroscope_left_foot_y', 'gyroscope_left_foot_z'],
        'left_shin': ['accelerometer_left_shin_x', 'accelerometer_left_shin_y', 'accelerometer_left_shin_z', 'gyroscope_left_shin_x', 'gyroscope_left_shin_y', 'gyroscope_left_shin_z'],
        'left_thigh': ['accelerometer_left_thigh_x', 'accelerometer_left_thigh_y', 'accelerometer_left_thigh_z', 'gyroscope_left_thigh_x', 'gyroscope_left_thigh_y', 'gyroscope_left_thigh_z']
    }
    
    orientations_dict = {sensor: [] for sensor in sensor_groups}

    # orientação inicial do quaternion, assumindo que não há rotação no início da leitura
    orientation = np.array([1.0, 0.0, 0.0, 0.0])

    # iteração entre todos os membros do df
    for i in range(len(df)):
        for sensor, columns in sensor_groups.items():
            # dados do acelerômetro e giroscópio, iterando sobre todos os grupos de sensores
            acc = np.array([df.iloc[i][columns[0]], df.iloc[i][columns[1]], df.iloc[i][columns[2]]])
            gyro = np.array([df.iloc[i][columns[3]], df.iloc[i][columns[4]], df.iloc[i][columns[5]]])
            
            # atualização do quaternion, utilizando o quaternion anterior
            orientation = cf.update(orientation, gyr=gyro, acc=acc)
            orientations_dict[sensor].append(orientation)

    # criação do df filtrado
    orientation_dfs = []
    for sensor, orientations in orientations_dict.items():
        orientation_df = pd.DataFrame(orientations, columns=[f'{sensor}_q0', f'{sensor}_q1', f'{sensor}_q2', f'{sensor}_q3'])
        orientation_dfs.append(orientation_df)

    return pd.concat(orientation_dfs, axis=1)

df = pd.read_csv('database.csv')

df_filtered = complementary_filter(df)
df_filtered['activity'] = df['activity'].values

df_filtered.head()

print(df_filtered.info())
print(df_filtered.head())

# Conversão do df em um único csv
df_filtered.to_csv('database_complementary_filter.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361344 entries, 0 to 361343
Data columns (total 25 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   right_foot_q0   361344 non-null  float64
 1   right_foot_q1   361344 non-null  float64
 2   right_foot_q2   361344 non-null  float64
 3   right_foot_q3   361344 non-null  float64
 4   right_shin_q0   361344 non-null  float64
 5   right_shin_q1   361344 non-null  float64
 6   right_shin_q2   361344 non-null  float64
 7   right_shin_q3   361344 non-null  float64
 8   right_thigh_q0  361344 non-null  float64
 9   right_thigh_q1  361344 non-null  float64
 10  right_thigh_q2  361344 non-null  float64
 11  right_thigh_q3  361344 non-null  float64
 12  left_foot_q0    361344 non-null  float64
 13  left_foot_q1    361344 non-null  float64
 14  left_foot_q2    361344 non-null  float64
 15  left_foot_q3    361344 non-null  float64
 16  left_shin_q0    361344 non-null  float64
 17  left_shin_